# Terremotos en Mexico

Proceso ETL:

+ Extraccion de datos desde el servicio sismologico nacional de la UNAM: http://www.ssn.unam.mx
+ Transformacion necesaria de los datos
+ Carga de datos en base de datos y mapa.

**Extraccion**

**Transformacion**

**Carga base datos de postgres**

**Mapa (heatmap)**

**Heatmap animado**